In [34]:
!curl -O "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4014  100  4014    0     0  10269      0 --:--:-- --:--:-- --:--:-- 10318


In [35]:
import minsearch

C:\Users\chetv\Documents\llm-zoomcamp\01-intro\minsearch.py:9: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [16]:
!curl -O  "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  642k  100  642k    0     0  1339k      0 --:--:-- --:--:-- --:--:-- 1344k


In [7]:
import json
import os

In [8]:
with open("documents.json", "rt") as file:
    docs_raw = json.load(file)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Minsearch (custom search using `TFIdf vectorizer`

In [36]:
# create index 
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=['course']
)

index.fit(documents)

In [37]:
question = "the course has already started, can i still enroll?"

In [38]:
# boosting some search fields
boost = {"question": 3.0, "section": 0.5}

# do the search
results = index.search(
    query=question,
    boost_dict=boost,
    num_results=5,
    filter_dict={"course": "data-engineering-zoomcamp"}
)    

In [39]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [40]:
import openai
from openai import OpenAI

In [41]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=OPENAI_API_KEY)

In [42]:
PROMPT_TEMPLATE = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
When answering the question, use only facts from the CONTEXT. If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [43]:
def build_context(search_results: list) -> str:
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    return context


def search(question: str,
           filter_dict={"course": "data-engineering-zoomcamp"},
           num_results = 5,
           boost={"question": 3.0, "section": 0.5})  -> list:

    search_results = index.search(
        query=question,
        boost_dict=boost,
        num_results=num_results,
        filter_dict=filter_dict
    )
    return search_results


def build_prompt(question: str) -> str: 
    search_results = search(question)
    context = build_context(search_results)
    return PROMPT_TEMPLATE.format(question=question, context=build_context(results))


def llm(prompt: str) -> str:
    response = client.chat.completions.create(
                model='gpt-4o',
                messages=[{"role": "user", "content": prompt}]
        )
    return response.choices[0].message.content

In [44]:
print(build_prompt(question))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
When answering the question, use only facts from the CONTEXT. If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can i still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

se

In [45]:
def rag(query: str) -> str:
    prompt = build_prompt(query)
    answer = llm(prompt)
    return answer

In [67]:
rag(question)

'Yes, even if the course has already started, you can still enroll and submit the homeworks. However, be aware of the deadlines for turning in the final projects.'

In [68]:
rag("how do i run kafka?")

'NONE'

# Elasticsearch
1. Run `elasticsearch` image with docker
2. Initialize client through forwarded port
3. Index settings
4. Run all docs to get them indexed
5. try some query

In [1]:
from elasticsearch import Elasticsearch

In [2]:
import elasticsearch

elasticsearch.__version__

(8, 4, 3)

In [3]:
# initialize Elasticsearch client
es_client = Elasticsearch("http://localhost:9200")

In [23]:
# index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

# index name to call for
index_name = "course-questions"

In [25]:
# create index
es_client.indices.create(index=index_name, body=index_settings)

In [13]:
from tqdm import tqdm

# index documents one by one
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████| 948/948 [00:53<00:00, 17.75it/s]


In [26]:
def elastic_search(query: str) -> list:
    # elastic query format
    search_query = {
        "size": 5, # numebr of results to return
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question, # our question
                        "fields": ["question^3", "text", "section"], # fields to check
                        "type": "best_fields"
                    }
                },
                "filter": { # filtering in only data engineering zoomcamp
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [27]:
question = "the course has already started, can i still enroll?"
elastic_search(question)

C:\Users\chetv\AppData\Local\Temp\ipykernel_19412\2328966626.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and 

In [46]:
def build_prompt_elastic(question: str) -> str: 
    search_results = elastic_search(question)
    context = build_context(search_results)
    return PROMPT_TEMPLATE.format(question=question, context=build_context(results))

def rag_elastic(query: str) -> str:
    prompt = build_prompt_elastic(query)
    answer = llm(prompt)
    return answer

In [47]:
rag_elastic(question)

C:\Users\chetv\AppData\Local\Temp\ipykernel_19412\2328966626.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, you can still enroll in the course after it has started. You are eligible to submit the homework even if you register late. However, keep in mind that there will be deadlines for submitting the final projects, so it's advisable not to delay everything until the last minute."